<a href="https://colab.research.google.com/github/suleymanovan/Analytical-projects/blob/main/FinalWorkBigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

Загружаем в колаб файлы по оценкам (u.data) и фильмам (u.items) и создаем на их основе pandas-датафреймы

In [ ]:
df_rat = pd.read_csv("u.data.csv", sep='\t', names=['user id', 'item id', 'rating', 'timestamp'], encoding="ISO-8859-1")

In [ ]:
df_rat.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
df_mov = pd.read_csv("u.item.csv", 
sep='|', header=None, names=['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 
'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
'Thriller', 'War', 'Western'], encoding="ISO-8859-1")

Добавим год релиза

In [ ]:
df_mov['release date'] = pd.to_datetime(df_mov['release date'])

In [ ]:
df_mov['video release date'] = df_mov['release date'].dt.year

In [ ]:
df_mov.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,1995.0,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,1995.0,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,1995.0,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1995-01-01,1995.0,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1995-01-01,1995.0,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


Средствами Pandas, используя dataframe ratings, найдем id пользователя, поставившего больше всего оценок

In [ ]:
df_id = df_rat.groupby('user id').count()

In [ ]:
df_id.sort_values(by=['rating'], ascending=False).iloc[0:1]

,item id,rating,timestamp
user id,,,
405,737,737,737


Оставим в датафрейме ratings только те фильмы, который оценил данный пользователь 

In [ ]:
df_filt= df_rat.loc[df_rat['user id'] == 405]

In [ ]:
df_filt.head()

,user id,item id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


Добавляем к датафрейму из задания 3 столбцы:
* По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет.
* Cтолбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей


In [ ]:
df = df_filt.merge(
    df_mov,
    left_on = 'item id',
    right_on = 'movie id',
    how = 'inner'
     )

In [ ]:
df.columns

Index(['user id', 'item id', 'rating', 'timestamp', 'movie id', 'movie title',
       'release date', 'video release date', 'IMDb URL', 'unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [ ]:
df = df[['item id', 'rating', 'video release date', 'unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

In [ ]:
df_agg = df_rat.groupby('item id')['rating'].agg(['sum', 'count'])
df_agg.head()

,sum,count
item id,,
1,1753,452
2,420,131
3,273,90
4,742,209
5,284,86


In [ ]:
df_all = df.merge(
    df_agg,
    left_on = 'item id',
    right_on = 'item id',
    how = 'inner'
     )

In [ ]:
df_all.head()

,item id,rating,video release date,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,sum,count
0,56,4,1994.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1600,394
1,592,1,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,30,9
2,1582,1,1947.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,171,1,1991.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,252,65
4,580,1,1995.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,108,32


Теперь все готово и можно строить модель. Сформируем X_train, X_test, y_train, y_test.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_all.columns

Index(['item id', 'rating', 'video release date', 'unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'sum', 'count'],
      dtype='object')

In [ ]:
X, y = df_all[['item id', 'video release date', 'unknown', 'Action', 'Adventure',
       'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'sum', 'count']], df_all['rating']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Приведем данные к одному масштабу при помощи StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Возьмем модель линейной регрессии и обучим ее на фильмах

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model.coef_

array([ 2.86760204e-02, -1.01668919e-01, -1.27675648e-15, -1.12248715e-01,
       -1.07081287e-01, -9.16493578e-02,  1.37354717e-01,  7.01882899e-02,
       -2.30578106e-03,  3.03353812e-02,  5.55311845e-02, -2.25434829e-02,
       -1.09640683e-01,  1.63775732e-03,  7.83019276e-02, -9.62120419e-03,
       -3.48840658e-02, -4.27076385e-03,  5.88587663e-02, -7.59178982e-02,
        5.04414636e-03, -7.99479213e-01,  1.55577112e+00])

Оценим качество модели на X_test, y_test при помощи метрик для задачи регрессии. Оцениваем качество с метрикой MSE (среднее отклонение предсказаний от правильных значений).

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model.predict(X_test)

array([1.51397529, 3.25684576, 1.23221489, 1.64303523, 1.3566551 ,
       1.26576079, 2.08567214, 1.17728415, 1.30475453, 1.54483389,
       2.66824123, 1.72351885, 1.3229466 , 1.5905652 , 1.81852409,
       2.04385981, 2.11548272, 1.21546795, 1.32293451, 3.84339422,
       1.34282044, 1.27868538, 2.0231106 , 2.0813822 , 1.48043758,
       1.24771484, 1.60617834, 1.83897447, 1.83069403, 0.74498353,
       3.82764557, 2.32551709, 1.30958885, 1.44006965, 1.2395798 ,
       1.94173651, 2.28023337, 1.15013082, 1.51357283, 1.11515082,
       1.42276808, 1.19757231, 1.21907827, 2.47662439, 1.46847376,
       2.00185687, 1.81408646, 1.98009558, 1.39991984, 1.23991106,
       2.77245405, 1.27054989, 1.48610112, 1.37499256, 4.87964479,
       1.32235663, 2.04096237, 3.06603271, 1.24272497, 1.28503255,
       1.49649894, 1.74621402, 1.22513601, 2.13612716, 1.78578341,
       1.20824144, 2.05738799, 1.42107502, 1.26241983, 2.03818802,
       3.16091657, 2.17548952, 2.56091512, 2.36193657, 1.29888

In [ ]:
mean_squared_error(y_train, model.predict(X_train))

1.1930637379734848

In [ ]:
mean_squared_error(y_test, model.predict(X_test))

1.1921541554627064

Загрузим данные в spark

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-2.4.7-bin-hadoop2.7.tgz

spark-2.4.7-bin-hadoop2.7/
spark-2.4.7-bin-hadoop2.7/kubernetes/
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/Dockerfile
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/Dockerfile
spark-2.4.7-bin-hadoop2.7/NOTICE
spark-2.4.7-bin-hadoop2.7/data/
spark-2.4.7-bin-hadoop2.7/data/strea

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import math

In [ ]:
df_rating = spark.read.csv('u.data.csv',  header=None, sep = '\t', inferSchema=True)

In [ ]:
df_rating = df_rating.toDF ( 'user id', 'movie id', 'rating', 'timestamp')

In [ ]:
df_rating.show(10)

+-------+--------+------+---------+
|user id|movie id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
+-------+--------+------+---------+
only showing top 10 rows



In [ ]:
df_movies = spark.read.csv('u.item.csv', inferSchema=True, header=None, sep='|')

In [ ]:
df_movies = df_movies.toDF ('movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 
'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
'Thriller', 'War', 'Western')

In [ ]:
df_movies.show(10)

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie id|         movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|    

Выведем среднюю оценку для каждого фильма

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_rat_avg = df_rating.groupBy('movie id').agg(F.count(df_rating.rating).alias('count'), F.avg(df_rating.rating).alias('avg'))

In [ ]:
df_rat_avg.show()

+--------+-----+------------------+
|movie id|count|               avg|
+--------+-----+------------------+
|     496|  231| 4.121212121212121|
|     471|  221|3.6108597285067874|
|     463|   71| 3.859154929577465|
|     148|  128|          3.203125|
|    1342|    2|               2.5|
|     833|   49| 3.204081632653061|
|    1088|   13| 2.230769230769231|
|    1591|    6|3.1666666666666665|
|    1238|    8|             3.125|
|    1580|    1|               1.0|
|    1645|    1|               4.0|
|     392|   68|3.5441176470588234|
|     623|   39| 2.923076923076923|
|     540|   43| 2.511627906976744|
|     858|    3|               1.0|
|     737|   59| 2.983050847457627|
|     243|  132|2.4393939393939394|
|    1025|   44|2.9318181818181817|
|    1084|   21| 3.857142857142857|
|    1127|   11| 2.909090909090909|
+--------+-----+------------------+
only showing top 20 rows



Получим 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок и по средней оценке)

In [ ]:
df_rat_avg.orderBy('count', ascending=False).show(5)

+--------+-----+------------------+
|movie id|count|               avg|
+--------+-----+------------------+
|      50|  583|4.3584905660377355|
|     258|  509|3.8035363457760316|
|     100|  508| 4.155511811023622|
|     181|  507| 4.007889546351085|
|     294|  485| 3.156701030927835|
+--------+-----+------------------+
only showing top 5 rows



In [ ]:
df_rat_avg.orderBy('count', ascending=True).show(5)

+--------+-----+---+
|movie id|count|avg|
+--------+-----+---+
|    1339|    1|1.0|
|    1507|    1|3.0|
|    1580|    1|1.0|
|    1460|    1|3.0|
|    1645|    1|4.0|
+--------+-----+---+
only showing top 5 rows



In [ ]:
df_rat_avg.orderBy('avg', ascending=False).show(5)

+--------+-----+---+
|movie id|count|avg|
+--------+-----+---+
|    1599|    1|5.0|
|    1653|    1|5.0|
|    1122|    1|5.0|
|    1500|    2|5.0|
|    1201|    1|5.0|
+--------+-----+---+
only showing top 5 rows



In [ ]:
df_rat_avg.orderBy('avg', ascending=True).show(5)

+--------+-----+---+
|movie id|count|avg|
+--------+-----+---+
|    1580|    1|1.0|
|    1352|    1|1.0|
|    1618|    1|1.0|
|     858|    3|1.0|
|    1339|    1|1.0|
+--------+-----+---+
only showing top 5 rows



Посчитаем среднюю оценку для каждого жанра

In [ ]:
df_join = df_rating.join(df_movies, on='movie id')

In [ ]:
df = df_join.select ('movie id', 'rating', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
'Thriller', 'War', 'Western')
df.show()

+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie id|rating|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     242|     3|      0|     0|        0|        0|        0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     302|     3|      0|     0|        0|        0|        0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
|     377|     1|      0|     0|        0|        0|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|   

In [ ]:
list_genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
'Thriller', 'War', 'Western'] 

In [ ]:
for Genre in list_genres:
    df_genre = df.where(df[Genre] == 1)
    df_genre.groupby(Genre).avg('rating').show()

+-------+-----------+
|unknown|avg(rating)|
+-------+-----------+
|      1|        3.2|
+-------+-----------+

+------+-----------------+
|Action|      avg(rating)|
+------+-----------------+
|     1|3.480245417953027|
+------+-----------------+

+---------+-----------------+
|Adventure|      avg(rating)|
+---------+-----------------+
|        1|3.503526503308369|
+---------+-----------------+

+---------+------------------+
|Animation|       avg(rating)|
+---------+------------------+
|        1|3.5766990291262135|
+---------+------------------+

+---------+------------------+
|Childrens|       avg(rating)|
+---------+------------------+
|        1|3.3532442216652742|
+---------+------------------+

+------+------------------+
|Comedy|       avg(rating)|
+------+------------------+
|     1|3.3940734781442745|
+------+------------------+

+-----+------------------+
|Crime|       avg(rating)|
+-----+------------------+
|    1|3.6322780881440098|
+-----+------------------+

+-----------+